In [1]:
import json
import torch

from tinynas.nn.networks import ProxylessNASNets
from utils import download_url

In [2]:
__all__ = ['net_id_list', 'build_model', 'download_tflite']

""" Note: all the memory and latency profiling is done with TinyEngine """
NET_INFO = {
    ##### imagenet models ######
    # mcunet models
    'mcunet-10fps': {
        'net_name': 'mcunet-10fps_imagenet',
        'description': 'MCUNet model that runs 10fps on STM32F746 (ImageNet)'
    },
    'mcunet-5fps': {
        'net_name': 'mcunet-5fps_imagenet',
        'description': 'MCUNet model that runs 5fps on STM32F746 (ImageNet)'
    },
    'mcunet-256kB': {
        'net_name': 'mcunet-256kb-1mb_imagenet',
        'description': 'MCUNet model that fits 256KB SRAM and 1MB Flash (ImageNet)',
    },
    'mcunet-320kB': {
        'net_name': 'mcunet-320kb-1mb_imagenet',
        'description': 'MCUNet model that fits 320KB SRAM and 1MB Flash (ImageNet)',
    },
    'mcunet-512kB': {
        'net_name': 'mcunet-512kb-2mb_imagenet',
        'description': 'MCUNet model that fits 512KB SRAM and 2MB Flash (ImageNet)',
    },
    # baseline models
    'mbv2-320kB': {
        'net_name': 'mbv2-w0.35-r144_imagenet',
        'description': 'scaled MobileNetV2 that fits 320KB SRAM and 1MB Flash (ImageNet)',
    },
    'proxyless-320kB': {
        'net_name': 'proxyless-w0.3-r176_imagenet',
        'description': 'scaled ProxylessNet that fits 320KB SRAM and 1MB Flash (ImageNet)'
    },

    ##### vww models ######
    'mcunet-10fps-vww': {
        'net_name': 'mcunet-10fps_vww',
        'description': 'MCUNet model that runs 10fps on STM32F746 (VWW)'
    },
    'mcunet-5fps-vww': {
        'net_name': 'mcunet-5fps_vww',
        'description': 'MCUNet model that runs 5fps on STM32F746 (VWW)'
    },
    'mcunet-320kB-vww': {
        'net_name': 'mcunet-320kb-1mb_vww',
        'description': 'MCUNet model that fits 320KB SRAM and 1MB Flash (VWW)'
    },

    ##### detection demo model ######
    'person-det': {
        'net_name': 'person-det',
        'description': 'person detection model used in our demo'
    },
}

In [3]:
net_id_list = list(NET_INFO.keys())

url_base = "https://hanlab.mit.edu/projects/tinyml/mcunet/release/"
net_id_list

['mcunet-10fps',
 'mcunet-5fps',
 'mcunet-256kB',
 'mcunet-320kB',
 'mcunet-512kB',
 'mbv2-320kB',
 'proxyless-320kB',
 'mcunet-10fps-vww',
 'mcunet-5fps-vww',
 'mcunet-320kB-vww',
 'person-det']

In [4]:
net_info = NET_INFO['mcunet-10fps']

net_config_url = url_base + net_info['net_name'] + ".json"
sd_url = url_base + net_info['net_name'] + ".pth"

net_config = json.load(open(download_url(net_config_url)))
resolution = net_config['resolution']

In [5]:
net_info

{'net_name': 'mcunet-10fps_imagenet',
 'description': 'MCUNet model that runs 10fps on STM32F746 (ImageNet)'}

In [6]:
net_config

{'name': 'ProxylessNASNets',
 'bn': {'momentum': 0.1, 'eps': 1e-05, 'ws_eps': None},
 'first_conv': {'name': 'ConvLayer',
  'kernel_size': 3,
  'stride': 2,
  'dilation': 1,
  'groups': 1,
  'bias': False,
  'has_shuffle': False,
  'in_channels': 3,
  'out_channels': 24,
  'use_bn': True,
  'act_func': 'relu6',
  'dropout_rate': 0,
  'ops_order': 'weight_bn_act'},
 'blocks': [{'name': 'MobileInvertedResidualBlock',
   'mobile_inverted_conv': {'name': 'MBInvertedConvLayer',
    'in_channels': 24,
    'out_channels': 16,
    'kernel_size': 3,
    'stride': 1,
    'expand_ratio': 1,
    'mid_channels': None,
    'act_func': 'relu6',
    'use_se': False},
   'shortcut': None},
  {'name': 'MobileInvertedResidualBlock',
   'mobile_inverted_conv': {'name': 'MBInvertedConvLayer',
    'in_channels': 16,
    'out_channels': 16,
    'kernel_size': 5,
    'stride': 2,
    'expand_ratio': 4,
    'mid_channels': 64,
    'act_func': 'relu6',
    'use_se': False},
   'shortcut': None},
  {'name': 'Mob

In [7]:
model = ProxylessNASNets.build_from_config(net_config)

In [8]:
model

ProxylessNASNets(
  (first_conv): ConvLayer(
    (conv): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU6(inplace=True)
  )
  (blocks): ModuleList(
    (0): MobileInvertedResidualBlock(
      (mobile_inverted_conv): MBInvertedConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=24, bias=False)
          (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU6(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(24, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
    (1): MobileInvertedResidualBlock(
      (mobile_inverted_conv): MBInvertedConvLayer(
        (inverted_b

In [9]:
import torch
a = torch.randn((1,3,256,256))

In [10]:
b = model(a)

In [11]:
for c in b:
    print(c.shape)

torch.Size([1, 24, 32, 32])
torch.Size([1, 56, 16, 16])
torch.Size([1, 192, 8, 8])


In [12]:
net_config_url

'https://hanlab.mit.edu/projects/tinyml/mcunet/release/mcunet-10fps_imagenet.json'

In [11]:
a = [1,2,3,4]

In [13]:
len(b)

15